In [134]:
import requests
import re
from bs4 import BeautifulSoup as bs

In [34]:
out_url_list = []
req=requests.get("https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000058563&action=getcompany&scd=filings")
text = req.text
soup=bs(text)
lis = soup.find_all(text='N-Q')

for i in lis:
    out_url_list.append('https://www.sec.gov' + i.next.next.a['href'])

In [35]:
url_list = []
for i in out_url_list:
    req = requests.get(i)
    text = req.text
    soup = bs(text)
    url_list.append('https://www.sec.gov'+ soup.find(text='AQR FUNDS').next.next.a['href'])

In [36]:
url_list

['https://www.sec.gov/Archives/edgar/data/1444822/000119312518334046/d628509dnq.htm',
 'https://www.sec.gov/Archives/edgar/data/1444822/000119312518174854/d555199dnq.htm',
 'https://www.sec.gov/Archives/edgar/data/1444822/000119312517355327/d486756dnq.htm']

In [413]:
soup_dic = dict()
time =['20170930', '20180331', '20180930']
for url in url_list:
    req = requests.get(url)
    text = req.text
    soup = bs(text)
    soup_dic[time.pop()] = soup
    year -= 1


In [610]:

def fun(day,country,dic):
    day = str(day)
    soup = soup_dic[day]
    country_td = soup.find('td', text=re.compile(country+'.*'))
    country_td_list = soup.find_all('td', text=re.compile(country_td.text.split('%')[0]+'.*'))
#     如果发现多个，但是没出现continued这个单词，就处理下
    if len(country_td_list)>1 and 'continued' not in country_td_list[1].text:
        country_td_list = [ country_td_list[0] ]
    print('='*50,'\n',day,[k.text for k in country_td_list],'\n')

    
    for country_td in country_td_list:
        country_tr = country_td.parent
        for i in country_tr.next_siblings:
        #     跳过不是tr的兄弟标签
            if i.name != 'tr':
                continue
            lis = i.text.split()
            
        #       当出现空的list退出循环,当一列有多个国家时会运行,这里是结束的地方
            if not lis:
#                 print('\n\n',dic, '\n\n')
                break

            if '$' in lis:
                lis.pop(lis.index('$'))

            company_name = ' '.join(lis[:-2])
#             把公司名里面的括号都去掉,然后去掉两边的空格
            company_name = company_name.split('(')[0].strip()
        #         把str的数字变成int格式
            company_invest_value_str = lis[-1]
            if ',' in company_invest_value_str:
                company_invest_value_str = company_invest_value_str.replace(',', '')

            company_invest_value = int(company_invest_value_str)
            
            print(company_name, company_invest_value)
#             如果字典里没有这个公司，那么就创建每年的都为零
            if not dic.get(company_name):
                dic[company_name] = {'20170930':0, '20180331':0, '20180930':0}
            dic[company_name][day] = company_invest_value



    

In [614]:
data_dic = {}
country_list = ['Belgium','Denmark','Finland','France','Germany']
for country in country_list:
    fun('20170930', country,data_dic)
    fun('20180331', country,data_dic)
    fun('20180930', country,data_dic)

 20170930 ['Belgium\xa0-\xa01.1%'] 

KBC Group NV 77544
Solvay SA 9867
UCB SA 37487
Umicore SA 3394
 20180331 ['Belgium - 0.6%'] 

Ageas 128495
Bekaert SA 35279
bpost SA 49207
KBC Group NV 285532
Solvay SA 109516
UCB SA 53912
Umicore SA 36871
 20180930 ['Belgium - 0.7%'] 

Ageas 960076
Colruyt SA 100112
KBC Group NV 425553
Solvay SA 231377
UCB SA 367723
Umicore SA 38877
 20170930 ['Denmark\xa0-\xa01.3%'] 

Danske Bank A/S 98020
Dfds A/S 972
GN Store Nord A/S 1133
H Lundbeck A/S 15841
Jyske Bank A/S 28848
Vestas Wind Systems A/S 10337
William Demant Holding A/S 5393
 20180331 ['Denmark - 0.4%'] 

Carlsberg A/S, Class B 101247
Danske Bank A/S 144703
Dfds A/S 11995
GN Store Nord A/S 90549
Jyske Bank A/S 31986
Novo Nordisk A/S, Class B 8411
Novozymes A/S, Class B 21917
Rockwool International A/S, Class B 35747
Tryg A/S 14329
William Demant Holding A/S 38032
 20180930 ['Denmark - 0.6%'] 

Carlsberg A/S, Class B 101696
Coloplast A/S, Class B 78864
Dfds A/S 24235
GN Store Nord A/S 284319
H Lu

In [616]:
rate_dic = {}
for k,v in data_dic.items():
    print(k,'\n' ,'invest value:', v)
    rate_dic[k] = {'20180331':'infinity', '20180930':'infinity' }
    
    try:
        rate_dic[k]['20180331'] = '{:.2%}'.format( (v['20180331']-v['20170930'])/v['20170930'] )
    except:
#         both of them are zero
        if v['20180331'] == v['20170930']:
            rate_dic[k]['20180331'] = '{:.2%}'.format(0)
    try:
        rate_dic[k]['20180930'] = '{:.2%}'.format((v['20180930']-v['20180331'])/v['20180331'])
    except:
#         both of them are zero
        if v['20180930'] == v['20180331']:
            rate_dic[k]['20180930'] = '{:.2%}'.format(0)
    print(' change rate: ',rate_dic[k],'\n')


KBC Group NV 
 invest value: {'20170930': 77544, '20180331': 285532, '20180930': 425553}
 change rate:  {'20180331': '268.22%', '20180930': '49.04%'} 

Solvay SA 
 invest value: {'20170930': 9867, '20180331': 109516, '20180930': 231377}
 change rate:  {'20180331': '1009.92%', '20180930': '111.27%'} 

UCB SA 
 invest value: {'20170930': 37487, '20180331': 53912, '20180930': 367723}
 change rate:  {'20180331': '43.82%', '20180930': '582.08%'} 

Umicore SA 
 invest value: {'20170930': 3394, '20180331': 36871, '20180930': 38877}
 change rate:  {'20180331': '986.36%', '20180930': '5.44%'} 

Ageas 
 invest value: {'20170930': 0, '20180331': 128495, '20180930': 960076}
 change rate:  {'20180331': 'infinity', '20180930': '647.17%'} 

Bekaert SA 
 invest value: {'20170930': 0, '20180331': 35279, '20180930': 0}
 change rate:  {'20180331': 'infinity', '20180930': '-100.00%'} 

bpost SA 
 invest value: {'20170930': 0, '20180331': 49207, '20180930': 0}
 change rate:  {'20180331': 'infinity', '20180